In [1]:
import numpy as np
import matplotlib.pyplot as plt
import autokeras as ak
import tensorflow as tf
import os

In [2]:
import tensorflow.keras as keras
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Dropout, Flatten, Convolution2D, MaxPooling2D , Lambda, Conv2D, Activation,Concatenate
from tensorflow.keras.layers import ActivityRegularization, BatchNormalization
from tensorflow.keras.optimizers import Adam , SGD , Adagrad
from tensorflow.keras.callbacks import ModelCheckpoint, LearningRateScheduler, EarlyStopping, CSVLogger, ReduceLROnPlateau
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import regularizers , initializers
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing.image import NumpyArrayIterator
from sklearn.preprocessing import StandardScaler

In [3]:
import logging

logging.basicConfig(level = logging.INFO)
gpus = tf.config.experimental.list_physical_devices('GPU')
# if gpus:
#   # Restrict TensorFlow to only use the first GPU
try:
    tf.config.experimental.set_visible_devices(gpus[0], 'GPU')
    tf.config.experimental.set_virtual_device_configuration(
    gpus[0],
    [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=2000)])
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
#     logging.info(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPU")
except RuntimeError as e:
# Visible devices must be set before GPUs have been initialized
    logging.info(e)

In [4]:
data = np.load("./nsi_data/sample_nsi_regression_1e7_v1.npz")

In [5]:
data_selection = 1 # 0 for all, 1 for lowE(<5GeV), 2 for high(>5GeV)

if data_selection == 0:
    data_all = np.column_stack([data['ve_dune'], data['vu_dune'], data['vebar_dune'], data['vubar_dune']])
elif data_selection == 1:
    data_all = np.column_stack([data['ve_dune'][:,:36], data['vu_dune'][:,:36], data['vebar_dune'][:,:36], data['vubar_dune'][:,:36]])
elif data_selection == 2:
    data_all = np.column_stack([data['ve_dune'][:,36:], data['vu_dune'][:,36:], data['vebar_dune'][:,36:], data['vubar_dune'][:,36:]])

pre_target = np.column_stack([data['theta23']/5,data['delta']/180+5,data['mumu']+5,data['emu']+5,data['etau']+5])
target = np.column_stack([pre_target, pre_target])

x_train = data_all[:10000]
y_train = target[:10000]
x_train2 = data_all[10000:9000000]
y_train2 = target[10000:9000000]
x_test = data_all[9000000:]
y_test = target[9000000:]

In [6]:
model_name = "nsi_degen_allparams"

In [7]:
from tensorflow.keras import backend as K

def custom_mse(y_true, y_pred):
    loss = K.square(y_pred - y_true)
    loss1 = loss[0]+loss[1]+loss[2]+loss[3]+loss[4]
    loss2 = loss[6]+loss[7]+loss[8]+loss[9]+loss[5]
    return K.switch(loss1<loss2, loss1, loss2)


def DNN_Model(name):
    model_DNN = Sequential(name = "Model_DNN_"+str(name))

    model_DNN.add(keras.Input(shape=(len(x_train[0]),), name = 'input'))
    model_DNN.add(BatchNormalization(name = 'BatchNormalization'))
    model_DNN.add(Dense(512, activation='relu', name = 'dense_1'))
    model_DNN.add(Dense(512, activation='relu', name = 'dense_2'))

    model_DNN.add(Dense(10, activation='relu', name = 'output'))
    model_opt = keras.optimizers.Adam()
    model_DNN.compile(loss=custom_mse,#keras.losses.binary_crossentropy
                              optimizer=model_opt,
                              metrics=['mse'])
    model_DNN.summary()
    return model_DNN

In [8]:
model = DNN_Model("regression")

Model: "Model_DNN_regression"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
BatchNormalization (BatchNor (None, 144)               576       
_________________________________________________________________
dense_1 (Dense)              (None, 512)               74240     
_________________________________________________________________
dense_2 (Dense)              (None, 512)               262656    
_________________________________________________________________
output (Dense)               (None, 10)                5130      
Total params: 342,602
Trainable params: 342,314
Non-trainable params: 288
_________________________________________________________________


In [9]:
batch_size = 2048
model.fit(x_train2, y_train2,
           validation_split = 0.1,
           batch_size=batch_size,
           epochs=20,
           verbose=1,
           shuffle = True
         )

Epoch 1/20
3951/3951 [==============================] - 15s 3ms/step - loss: 5.2567 - mse: 1.4724 - val_loss: 0.9013 - val_mse: 0.2702
Epoch 2/20
3951/3951 [==============================] - 13s 3ms/step - loss: 0.9716 - mse: 0.3001 - val_loss: 0.9360 - val_mse: 0.2949
Epoch 3/20
3951/3951 [==============================] - 13s 3ms/step - loss: 0.7353 - mse: 0.2304 - val_loss: 0.5919 - val_mse: 0.1854
Epoch 4/20
3951/3951 [==============================] - 13s 3ms/step - loss: 0.5531 - mse: 0.1865 - val_loss: 0.5333 - val_mse: 0.1833
Epoch 5/20
3951/3951 [==============================] - 13s 3ms/step - loss: 0.4559 - mse: 0.1581 - val_loss: 0.4104 - val_mse: 0.1446
Epoch 6/20
3951/3951 [==============================] - 13s 3ms/step - loss: 0.3705 - mse: 0.1346 - val_loss: 0.3213 - val_mse: 0.1179
Epoch 7/20
3951/3951 [==============================] - 12s 3ms/step - loss: 0.3266 - mse: 0.1211 - val_loss: 0.2727 - val_mse: 0.1056
Epoch 8/20
3951/3951 [==============================] -

In [10]:
model_index = 1
while os.path.isfile('./models/{}_{}.h5'.format(model_name, model_index)):
    model_index += 1
model.save('./models/{}_{}.h5'.format(model_name, model_index))

In [11]:
scale_steps = np.logspace(-3, 0, 30)
before_train_loss = []
after_train_loss = []

for scale in scale_steps:
    x_train2_gen = np.random.normal(x_train2, np.sqrt(x_train2)*scale)
    x_test_gen = np.random.normal(x_test, np.sqrt(x_test)*scale)

    before_train_loss.append(model.evaluate(x_test_gen, y_test)[0])

    model.fit(x_train2_gen, y_train2,
               validation_split = 0.1,
               batch_size=batch_size,
               epochs=5,
               verbose=1,
               shuffle = True
             )
    
    after_train_loss.append(model.evaluate(x_test_gen, y_test)[0])

31250/31250 [==============================] - 53s 2ms/step - loss: 0.1541 - mse: 0.0685
Epoch 1/5
3951/3951 [==============================] - 13s 3ms/step - loss: 0.1457 - mse: 0.0652 - val_loss: 0.1431 - val_mse: 0.0637
Epoch 2/5
3951/3951 [==============================] - 13s 3ms/step - loss: 0.1416 - mse: 0.0632 - val_loss: 0.1411 - val_mse: 0.0641
Epoch 3/5
3951/3951 [==============================] - 13s 3ms/step - loss: 0.1368 - mse: 0.0613 - val_loss: 0.1268 - val_mse: 0.0601
Epoch 4/5
3951/3951 [==============================] - 13s 3ms/step - loss: 0.1364 - mse: 0.0608 - val_loss: 0.1666 - val_mse: 0.0678
Epoch 5/5
31250/31250 [==============================] - 55s 2ms/step - loss: 0.1347 - mse: 0.0607
Epoch 1/5
3951/3951 [==============================] - 13s 3ms/step - loss: 0.1309 - mse: 0.0589 - val_loss: 0.1150 - val_mse: 0.0581
Epoch 2/5
3951/3951 [==============================] - 13s 3ms/step - loss: 0.1257 - mse: 0.0575 - val_loss: 0.0972 - val_mse: 0.0484
Epoch 3/

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



31250/31250 [==============================] - 53s 2ms/step - loss: 0.0747 - mse: 0.0395
Epoch 1/5
3951/3951 [==============================] - 15s 4ms/step - loss: 0.0790 - mse: 0.0402 - val_loss: 0.0750 - val_mse: 0.0378
Epoch 2/5
3951/3951 [==============================] - 13s 3ms/step - loss: 0.0762 - mse: 0.0402 - val_loss: 0.0622 - val_mse: 0.0395
Epoch 3/5
3951/3951 [==============================] - 13s 3ms/step - loss: 0.0778 - mse: 0.0402 - val_loss: 0.0696 - val_mse: 0.0378
Epoch 4/5
3951/3951 [==============================] - 13s 3ms/step - loss: 0.0742 - mse: 0.0393 - val_loss: 0.0676 - val_mse: 0.0380
Epoch 5/5
31250/31250 [==============================] - 54s 2ms/step - loss: 0.0799 - mse: 0.0439
Epoch 1/5
3951/3951 [==============================] - 13s 3ms/step - loss: 0.0775 - mse: 0.0402 - val_loss: 0.0854 - val_mse: 0.0448
Epoch 2/5
3951/3951 [==============================] - 12s 3ms/step - loss: 0.0797 - mse: 0.0412 - val_loss: 0.0729 - val_mse: 0.0404
Epoch 3/

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



3951/3951 [==============================] - 12s 3ms/step - loss: 0.0918 - mse: 0.0467 - val_loss: 0.0865 - val_mse: 0.0435
Epoch 4/5
3951/3951 [==============================] - 12s 3ms/step - loss: 0.0885 - mse: 0.0463 - val_loss: 0.0910 - val_mse: 0.0482
Epoch 5/5
31250/31250 [==============================] - 53s 2ms/step - loss: 0.0979 - mse: 0.0482
Epoch 1/5
3951/3951 [==============================] - 13s 3ms/step - loss: 0.0987 - mse: 0.0510 - val_loss: 0.0961 - val_mse: 0.0489
Epoch 2/5
2210/3951 [===============>..............] - ETA: 4s - loss: 0.0990 - mse: 0.0518

...

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



31250/31250 [==============================] - 53s 2ms/step - loss: 0.2115 - mse: 0.0894
Epoch 1/5
3951/3951 [==============================] - 13s 3ms/step - loss: 0.1993 - mse: 0.0865 - val_loss: 0.2003 - val_mse: 0.0879
Epoch 2/5
3951/3951 [==============================] - 12s 3ms/step - loss: 0.1928 - mse: 0.0835 - val_loss: 0.1837 - val_mse: 0.0830
Epoch 3/5
3951/3951 [==============================] - 12s 3ms/step - loss: 0.1893 - mse: 0.0820 - val_loss: 0.2022 - val_mse: 0.0840
Epoch 4/5
3951/3951 [==============================] - 12s 3ms/step - loss: 0.1864 - mse: 0.0825 - val_loss: 0.1737 - val_mse: 0.0782
Epoch 5/5
17166/31250 [===============>..............] - ETA: 23s - loss: 0.2166 - mse: 0.0931

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



3951/3951 [==============================] - 12s 3ms/step - loss: 0.2693 - mse: 0.1072 - val_loss: 0.2686 - val_mse: 0.1047
Epoch 5/5
31250/31250 [==============================] - 52s 2ms/step - loss: 0.2921 - mse: 0.1180
Epoch 1/5
3951/3951 [==============================] - 13s 3ms/step - loss: 0.3091 - mse: 0.1204 - val_loss: 0.2930 - val_mse: 0.1173
Epoch 2/5
3951/3951 [==============================] - 12s 3ms/step - loss: 0.3027 - mse: 0.1198 - val_loss: 0.2944 - val_mse: 0.1136
Epoch 3/5
2055/3951 [==============>...............] - ETA: 5s - loss: 0.3039 - mse: 0.1184

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



In [12]:
furthur_index = 1
path = './models_furthurTrain/{}_{}_{}.h5'
while os.path.isfile(path.format(model_name, model_index, furthur_index)):
    index += 1
model.save(path.format(model_name, model_index, furthur_index))
outfile = {'scale_steps': scale_steps,
           'before_train_loss': before_train_loss,
           'after_train_loss' :after_train_loss}
np.save(file = './models_furthurTrain/{}_{}_{}_result.npy'.format(model_name, model_index, furthur_index),
        arr = outfile)

In [13]:
x_test2_gen = np.random.poisson(x_test)

for i in range(10):
    x_train2_gen = np.random.poisson(x_train2)
    
    model.fit(x_train2_gen, y_train2,
              validation_split=0.1,
               batch_size=batch_size,
               epochs=1,
               verbose=1,
               shuffle = True
             )
model.evaluate(x_test2_gen, y_test)

 7960/31250 [======>.......................] - ETA: 43s - loss: 0.3476 - mse: 0.1351

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



In [14]:
index = 1
path = './models_PoissonTrain/{}_{}_{}_{}.h5'
while os.path.isfile(path.format(model_name, model_index, furthur_index, index)):
    index += 1
model.save(path.format(model_name, model_index, furthur_index, index))